## AMAZION Web scraping an notifier project

In [10]:
#importing the libraries
from bs4 import BeautifulSoup
import requests
import time
import datetime


### Step 1: connecting to the web URL and retrieving the HTML data

In [12]:
#Connecting to the website
URL='https://www.amazon.nl/Koffiepadapparaat-Koffievariaties-Waterreservoir-Intensiteit-90/dp/B08Z4B4LKF/ref=sr_1_1'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page= requests.get(URL, headers=headers)

soup1=BeautifulSoup(page.content,"html.parser")
soup2=BeautifulSoup(soup1.prettify(),"html.parser")



### Step 2: Finding the product title, price and current date 

In [14]:
#getting the title from the retrieved html data
title= soup2.find(id='productTitle').get_text()
title=title.strip()
print(title)

Philips Senseo Original Plus Koffiepadapparaat -2 Koffievariaties (Mild en Intens) -Zet 1 of 2 Kopjes Tegelijk -0.7 Liter Waterreservoir -Matte afwerking -Intensiteit selectie -Rood -CSA210/90


In [16]:
#getting the price from the retrieved html data
price_whole = soup2.find('span', class_='a-price-whole')
price_fraction = soup2.find('span', class_='a-price-fraction')
price_symbol = soup2.find('span', class_='a-price-symbol')

if price_whole and price_fraction and price_symbol:
    price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
else:
    print("Price element not found")

print(price)



€57
                              
                               ,.09


In [18]:
#formatting the price
price = price.replace('\n', '').replace(' ', '').replace(',', '').strip()[1:]
print(price)

57.09


In [20]:
import datetime
today=datetime.date.today()
print(today)

2024-07-01


### Step 3: Writing the data into a CSV file

In [22]:
#writing the data into a csv
import csv
header =['Title','Price','Date']
data=[title,price,today]
with open('AmazonScraper.csv','w', newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)
    

In [24]:
#reading the data from a csv 
import pandas as pd 
df=pd.read_csv(r'C:\Users\User\Data_python\AmazonScraper.csv')
print(df)

                                               Title  Price        Date
0  Philips Senseo Original Plus Koffiepadapparaat...  57.09  2024-07-01


### Step 4: Writing a fuction for email notification

In [26]:
import os
import smtplib
from dotenv import load_dotenv



def send_mail():
    # Get the email and password from environment variables
    # Load variables from .env file
    load_dotenv()

    # Retrieve environment variables
    email_user = os.getenv('EMAIL_USER')
    email_pass = os.getenv('EMAIL_PASS')


    if not email_user or not email_pass:
        raise ValueError("Email user or password not set in environment variables")

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(email_user, email_pass)
    
    subject = "The coffee machine you want is below $50! Now is your chance to buy!"
    body = "Tanvir, This is the moment we have been waiting for. Now is your chance to pick up the coffee machine of your dreams. Don't mess it up! Link here: https://www.amazon.nl/Koffiepadapparaat-Koffievariaties-Waterreservoir-Intensiteit-90/dp/B08Z4B4LKF/ref=sr_1_1"
   
    msg = f"Subject: {subject}\n\n{body}"
    recipient_email = 'raining.smtt@gmail.com'  # Replace with the actual recipient email address

    server.sendmail(
        email_user,
        recipient_email,
        msg
    )


### Step 5: Putting the Step 1-3 in a single function (price_check) and invoking step 4 (send_email) if price drops below a certain point

In [29]:

def price_check():
    URL='https://www.amazon.nl/Koffiepadapparaat-Koffievariaties-Waterreservoir-Intensiteit-90/dp/B08Z4B4LKF/ref=sr_1_1'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page= requests.get(URL, headers=headers)

    soup1=BeautifulSoup(page.content,"html.parser")
    soup2=BeautifulSoup(soup1.prettify(),"html.parser")

    #getting the title
    title= soup2.find(id='productTitle').get_text()
    title=title.strip()
    #getting the price
    price_whole = soup2.find('span', class_='a-price-whole')
    price_fraction = soup2.find('span', class_='a-price-fraction')
    price_symbol = soup2.find('span', class_='a-price-symbol')

    if price_whole and price_fraction and price_symbol:
        price = f"{price_symbol.get_text().strip()}{price_whole.get_text().strip()}.{price_fraction.get_text().strip()}"
    else:
        price='N/A'

    #formatting the price
    price = price.replace('\n', '').replace(' ', '').replace(',', '').strip()[1:]

    import datetime
    today=datetime.date.today()
    print(today)

    import csv
    header =['Title','Price','Date']
    data=[title,price,today]
    with open('AmazonScraper.csv','a+', newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)
    price= float(price)
    if(price<50.00):
        send_mail()

### Step 6: Running step 5 once every day automatically

In [ ]:
while(True):
    price_check()
    time.sleep(86400)

2024-07-01
